In [1]:
import ollama
import gradio as gr

MODEL="llama2"

c:\Users\shahe\Documents\SuperDataScience\Collaborative Projects\Ongoing\SDS-CP019-local-ai-writing-assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
messages = [
    {"role":"assistant","content":"You are an unhelpful assistant and will respond to user queries with sarcasm"},
    {"role":"user","content":"What is the capital of france"}
]

In [3]:
response = ollama.chat(model=MODEL, messages=messages)

In [ ]:
print(response['message']['content'])

## Building a chatbot

In [ ]:
# Simple chat code logic

def simple_chat():

    system_message = "You are an unhelpful assistant and will respond to user queries with sarcasm"
    print("Hello. I am your personal unhelpful assistant. How can I help you today?")
    
    user_input = input("You: ")

    while user_input != '/bye':
        messages.append({"role":"user","content":user_input})
        resp = ollama.chat(model=MODEL, messages=messages)
        messages.append({"role":"assistant","content":resp['message']['content']})
        print(resp['message']['content'])
        user_input = input("You: ")

    return print("You have exited the chat!")


In [2]:
# Chat function for Gradio

def chat_llama2(prompt, history):
    system_message = "You are an unhelpful assistant and will respond to user queries with sarcasm"
    
    messages = [
        {"role":"assistant","content":system_message},
        {"role":"user","content":prompt}
    ]

    result = ollama.chat(
        model="llama2",
        messages=messages,
        stream=True         # allows us to stream the resulting response instead of displaying the entire output at once
        )
    
    # streaming the response from the model
    # this prints out each output token form the model to have the resulting text displayed in real time
    response = ""
    for chunk in result:
        if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

### Gradio UI with a high level wrapper (ChatInterface)

In [ ]:
gr.ChatInterface(
    fn=chat_llama2,
    type="messages",
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="Unhelpful Assistant",
    description="Ask me a question and I will give you an unhelpful answer",
    theme="ocean",
    examples=["Are tomatoes a vegetable", "What is the capital of France", "Is yes no and a no a yes"]
    # cache_examples=True
    ).launch()

# Scrapping website to create brochure

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://hashpotato.io/"

In [4]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
# print(soup.body("nav"))

In [4]:
class Website:
    """
    A utility class to represent the website to scrape
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # removing irrelevant information
        if soup.body:
            for irr in soup.body(["script","style","img","input"]):
                irr.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [20]:
website = Website(url)

In [ ]:
website.text

In [5]:
def get_url_from_prompt(prompt):

    words = [word for word in prompt.split(" ")]
    url = [word for word in words if 'http' in word]

    return url

In [6]:
def brochure_bot(prompt, history):

    system_prompt = "You are a creative writing assistant"
    system_prompt += " and your job is to help write a brochure for the company that the user provides."
    system_prompt += " Make sure to keep your responses professional and formal."
    system_prompt += " If you do not know information to a question the user provides,"
    system_prompt += " mention clearly that you do not know the answer."
    system_prompt += " Your first task is to ask the user to give you the URL of a website."
    system_prompt += " If you do not find one, ask the user for one."

    url = get_url_from_prompt(prompt)
    try:
        website_info = Website(url[0]).text
        prompt += website_info
    except:
        prompt += "No URL provided"

    messages = [
        {"role":"assistant", "content":system_prompt},
        {"role":"user", "content":prompt}
    ]

    result = ollama.chat(
        model="llama2",
        messages=messages,
        stream=True         # allows us to stream the resulting response instead of displaying the entire output at once
        )
    
    # streaming the response from the model
    # this prints out each output token form the model to have the resulting text displayed in real time
    response = ""
    for chunk in result:
        if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

In [7]:
# using the same interface as before for the time being
gr.ChatInterface(
    fn=brochure_bot,
    type="messages",
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="Brochure bot",
    description="Give me the link to a website for which you want to generate a brochure",
    theme="ocean"
    # examples=["Are tomatoes a vegetable", "What is the capital of France", "Is yes no and a no a yes"]
    # cache_examples=True
    ).launch()

c:\Users\shahe\Documents\SuperDataScience\Collaborative Projects\Ongoing\SDS-CP019-local-ai-writing-assistant\.venv\Lib\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
c:\Users\shahe\Documents\SuperDataScience\Collaborative Projects\Ongoing\SDS-CP019-local-ai-writing-assistant\.venv\Lib\site-packages\gradio\chat_interface.py:222: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
